In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, expon, lognorm, entropy, weibull_min

class KLModelEvaluator:
    def __init__(self, observed_data, bins=40):
        self.observed_data = observed_data
        self.bins = np.histogram_bin_edges(observed_data, bins=bins)
        self.observed_hist, _ = np.histogram(observed_data, bins=self.bins, density=True)
        self.observed_midpoints = (self.bins[:-1] + self.bins[1:]) / 2
        self.models = {}
        self.kl_results = {}

    def add_model(self, name, distribution, fit_args=None):
        fit_args = fit_args or {}
        params = distribution.fit(self.observed_data, **fit_args)
        pdf = distribution.pdf(self.observed_midpoints, *params)
        self.models[name] = pdf
        self.kl_results[name] = self._kl_divergence(self.observed_hist, pdf)

    def _kl_divergence(self, p, q):
        epsilon = 1e-10
        p = np.clip(p, epsilon, None)
        q = np.clip(q, epsilon, None)
        return entropy(p, q)

    def plot_results(self):
        num_models = len(self.models)
        rows = (num_models + 1) // 2
        fig, axes = plt.subplots(rows, 2, figsize=(12, 5 * rows))
        axes = axes.flatten()
        
        for i, (name, pdf) in enumerate(self.models.items()):
            axes[i].hist(self.observed_data, bins=self.bins, density=True, alpha=0.6, color="skyblue", label="Observed Distribution")
            axes[i].plot(self.observed_midpoints, pdf, 'r-', label=f"{name} Distribution")
            axes[i].set_title(f"KL Divergence: {self.kl_results[name]:.2f} ({name})")
            axes[i].set_xlabel("Value")
            axes[i].set_ylabel("Density")
            axes[i].legend()

        plt.tight_layout()
        plt.show()

    def print_results(self):
        print("KL Divergence Results:")
        for name, kl in self.kl_results.items():
            print(f"{name}: {kl:.2f}")
        best_model = min(self.kl_results, key=self.kl_results.get)
        print(f"\nThe most aligned model is: {best_model} with KL Divergence = {self.kl_results[best_model]:.2f}")

if __name__ == "__main__":
    # Generate observed data (complex simulated data)
    np.random.seed(42)
    data1 = np.random.normal(loc=5, scale=1.5, size=500)
    data2 = np.random.exponential(scale=2.0, size=300)
    data3 = np.random.uniform(low=2, high=6, size=200)
    observed_data = np.concatenate([data1, data2, data3])

    # Initialize evaluator
    evaluator = KLModelEvaluator(observed_data)

    # Add models
    evaluator.add_model("Gaussian", norm)
    evaluator.add_model("Exponential", expon)
    evaluator.add_model("Log-Normal", lognorm, fit_args={"floc": 0})
    evaluator.add_model("Weibull", weibull_min, fit_args={"floc": 0})

    # Plot results
    evaluator.plot_results()

    # Print KL divergence results
    evaluator.print_results()
